In [ ]:
#check CUDA und d3rlpy Installation
!nvidia-smi
!nvcc --version
import torch 
print(torch.version.cuda)  # sollte '12.2' oder ähnlich anzeigen
print(torch.cuda.is_available())  # sollte True sein
print(torch.cuda.current_device())  # sollte 0 sein, wenn eine GPU verfügbar ist
import d3rlpy
print(d3rlpy.__version__)

12.1
True
0


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.8.1
Mon Nov 10 12:49:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 539.56                 Driver Version: 539.56       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P4                     TCC   | 00000000:06:10.0 Off |                    0 |
| N/A   32C    P8               6W /  75W |      8MiB /  7680MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-------------------

In [ ]:
import d3rlpy
import numpy as np


# vector observation
# 1000 steps of observations with shape of (100,)
observations = np.random.random((1000, 100))

# 1000 steps of actions with shape of (4,)
actions = np.random.random((1000, 4))

# 1000 steps of rewards
rewards = np.random.random(1000)

# 1000 steps of terminal flags
terminals = np.random.randint(2, size=1000)

dataset = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)


# if you don't use GPU, set device=None instead.
sac = d3rlpy.algos.SACConfig().create(device='cuda:0')

# initialize neural networks with the given observation shape and action size.
# this is not necessary when you directly call fit or fit_online method.
sac.build_with_dataset(dataset)
# calculate metrics with training dataset
td_error_evaluator = d3rlpy.metrics.TDErrorEvaluator(episodes=dataset.episodes)

# evaluate algorithm on the environment
#rewards = env_evaluator(sac, dataset=None)

sac.fit(
    dataset,
    n_steps=10000,
    evaluators={
        'td_error': td_error_evaluator,
    }
)

# get first observation from the dataset
observation = observations[0]

# return actions based on the greedy-policy
action = sac.predict(np.expand_dims(observation, axis=0))

# estimate action-values
value = sac.predict_value(np.expand_dims(observation, axis=0), action)

# save full parameters and configurations in a single file.
sac.save('sac.d3')
# load full parameters and build algorithm
sac2 = d3rlpy.load_learnable("sac.d3")

# save full parameters only
sac.save_model('sac.pt')

# save the greedy-policy as TorchScript
sac.save_policy('policy.pt')

2025-11-10 12:49.55 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(4,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(100,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-11-10 12:49.55 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2025-11-10 12:49.55 [info     ] Action size has been automatically determined. action_size=4
2025-11-10 12:49.56 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(100,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(4,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=4)
2025-11-10 12:49.56 [warning  ] Skip building models since they're already built.
2025-11-10 12:49.57 [info     ] Directory is created at d3rlp

Epoch 1/1: 100%|██████████| 10000/10000 [04:40<00:00, 35.69it/s, critic_loss=8.45, actor_loss=-93.2, temp=1.24, temp_loss=-1.66]    


2025-11-10 12:54.38 [info     ] SAC_20251110124957: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.006094509816169739, 'time_algorithm_update': 0.021029426193237305, 'critic_loss': 8.500654422685503, 'actor_loss': -93.52499094626903, 'temp': 1.2426019729018212, 'temp_loss': -1.6740991617945022, 'time_step': 0.027544213247299193, 'td_error': 5650.247548962815} step=10000
2025-11-10 12:54.38 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251110124957\model_10000.d3


c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\d3rlpy\torch_utility.py:431: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chkpt = torch.load(f, map_locatio